# Trainer NNX scratchpad

Generate training data
```
python rgi/main.py --game connect4 --player1 random --player2 random --num_games 100 --save_trajectories
```

In [67]:
import os
from typing import Any

from collections import Counter

import jax
import jax.numpy as jnp
from flax import nnx

print(jax.devices())
assert jax.devices()[0].platform == 'gpu'

# print wider lines to stop arrays wrapping so soon. numpy default is 75.
jnp.set_printoptions(linewidth=150)

[CudaDevice(id=0)]


# Load Trajectories

In [68]:
from rgi.core import trajectory

game_name = "connect4"
trajectories_glob = os.path.join("..", "data", "trajectories", game_name, "*.trajectory.npy")
trajectories = trajectory.load_trajectories(trajectories_glob)

print(f'trajectories_glob: {trajectories_glob}')
print(f'num_trajectories: {len(trajectories)}')

trajectories_glob: ../data/trajectories/connect4/*.trajectory.npy
num_trajectories: 100


In [69]:
def print_trajectory_stats():
    c = Counter((t.actions[0].item(), t.final_rewards[0].item()) for t in trajectories)
    for action in range(1,7+1):
        win_count, loss_count = c[(action, 1.0)], c[(action, -1.0)]
        print(f'action: {action}, win: {win_count:3d}, loss: {loss_count:3d}, win_pct: {win_count / (win_count + loss_count) * 100:.2f}%')

print_trajectory_stats()

# num_trajectories: 100
# action: 1, win:  14, loss:   9, win_pct: 60.87%
# action: 2, win:   6, loss:   6, win_pct: 50.00%
# action: 3, win:   8, loss:   4, win_pct: 66.67%
# action: 4, win:  11, loss:   3, win_pct: 78.57%
# action: 5, win:  11, loss:   5, win_pct: 68.75%
# action: 6, win:   8, loss:   5, win_pct: 61.54%
# action: 7, win:   5, loss:   5, win_pct: 50.00%

action: 1, win:  14, loss:   9, win_pct: 60.87%
action: 2, win:   6, loss:   6, win_pct: 50.00%
action: 3, win:   8, loss:   4, win_pct: 66.67%
action: 4, win:  11, loss:   3, win_pct: 78.57%
action: 5, win:  11, loss:   5, win_pct: 68.75%
action: 6, win:   8, loss:   5, win_pct: 61.54%
action: 7, win:   5, loss:   5, win_pct: 50.00%


In [70]:
from rgi.core.trajectory import EncodedTrajectory

# define trajectory NamedTuple
from typing import NamedTuple
class TrajectoryStep(NamedTuple):
    move_index: int
    state: jax.Array
    action: jax.Array
    next_state: jax.Array
    reward: jax.Array


def unroll_trajectory(encoded_trajectories: list[EncodedTrajectory]):
    for t in encoded_trajectories:
        for i in range(t.length - 1):
            yield TrajectoryStep(i, t.states[i], t.actions[i], t.states[i + 1], t.state_rewards[i])

all_trajecoty_steps = list(unroll_trajectory(trajectories))
print(f'num_trajectory_steps: {len(all_trajecoty_steps)}')

num_trajectory_steps: 2193


In [71]:
state_batch = jnp.array([t.state for t in all_trajecoty_steps])
action_batch = jnp.array([t.action for t in all_trajecoty_steps])
reward_batch = jnp.array([t.reward for t in all_trajecoty_steps])
batch_full = {'state': state_batch, 'action': action_batch, 'reward': reward_batch}

# batch with 1 or 2 steps for testing
batch_1 = {'state': state_batch[:1], 'action': action_batch[:1], 'reward': reward_batch[:1]}
batch_2 = {'state': state_batch[:2], 'action': action_batch[:2], 'reward': reward_batch[:2]}

# Super easy to get 100% accuracy.
batch_repeated = {'state': jnp.tile(batch_2['state'], (1000,1)), 'action': jnp.tile(batch_2['action'], (1000,)), 'reward': jnp.tile(batch_2['reward'], (1000,))}

# Define models

In [72]:
from rgi.games import connect4
game = connect4.Connect4Game()
serializer = connect4.Connect4Serializer()

In [73]:
class Connect4StateEmbedder(nnx.Module):
    embedding_dim: int = 64
    hidden_dim: int = 256

    def _state_to_array(self, encoded_state_batch: jax.Array) -> jax.Array:
        board_array = encoded_state_batch[:, :-1].reshape([-1, 6, 7])
        return board_array

    def __init__(self, *, rngs: nnx.Rngs):
        self.conv1 = nnx.Conv(1, 32, kernel_size=(3, 3), rngs=rngs)
        self.conv2 = nnx.Conv(32, 64, kernel_size=(3, 3), rngs=rngs)
        self.linear1 = nnx.Linear(6*7*64, self.hidden_dim, rngs=rngs)
        self.linear2 = nnx.Linear(self.hidden_dim, self.embedding_dim, rngs=rngs)

    def __call__(self, encoded_state_batch: jax.Array):
        x = self._state_to_array(encoded_state_batch)
        x = x[..., None]  # Add channel dimension
        x = nnx.relu(self.conv1(x))
        x = nnx.relu(self.conv2(x))
        x = x.reshape(x.shape[0], -1)  # Flatten while preserving batch dimension
        x = nnx.relu(self.linear1(x))
        x = self.linear2(x)

        return x


In [74]:
def test_state_embedder():
    state_embedder = Connect4StateEmbedder(rngs=nnx.Rngs(0))
    print(state_embedder(trajectories[0].states[:1]))

test_state_embedder()

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [75]:
class Connect4ActionEmbedder(nnx.Module):
    embedding_dim: int = 64
    num_actions: int = 7

    def __init__(self, *, rngs: nnx.Rngs):
        self.embedding = nnx.Embed(num_embeddings=self.num_actions, features=self.embedding_dim, rngs=rngs)

    def __call__(self, action: jax.Array) -> jax.Array:
        # Ensure action is 0-indexed
        action = action - 1
        return self.embedding(action)
    
    def all_action_embeddings(self):
        # return self(jnp.arange(1,self.num_actions+1))
        return self.embedding.embedding.value

In [76]:
def test_action_embedder():
    action_embedder = Connect4ActionEmbedder(rngs=nnx.Rngs(0))
    print(action_embedder(trajectories[0].actions[:1]))
    print(action_embedder.all_action_embeddings()[:,:5])    

test_action_embedder()

[[ 6.66427687e-02  1.09368816e-01 -1.89152256e-01  8.17345604e-02 -7.85972252e-02 -1.82855129e-01  1.57606214e-01  1.16470948e-01 -7.91573077e-02
   1.19286872e-01 -3.06055341e-02 -2.94262134e-02 -2.04068035e-01  2.47045040e-01 -2.14104000e-02 -1.46370962e-01  3.72717567e-02  6.29766062e-02
  -4.52014022e-02 -2.30931133e-01  1.21301249e-01  2.88546652e-01 -3.97264175e-02 -1.19131254e-02  1.30750760e-01  1.29211560e-01  3.51449512e-02
   1.26254661e-02  7.07970336e-02  1.36475870e-02  3.25598754e-02  5.28224790e-03 -7.92904049e-02  2.16124147e-01  1.23112321e-01 -5.16295396e-02
  -5.90516999e-03 -7.26998299e-02  1.06619947e-01  4.65621985e-02  1.09553948e-01 -3.07245195e-01  6.44185543e-02 -5.80371059e-02  1.00223973e-01
  -4.86722216e-02  2.83067394e-02  2.01851264e-01  3.05441283e-02 -9.47859734e-02 -8.36363509e-02 -1.35540087e-02 -5.33435494e-02  7.62056708e-02
   8.15421045e-02  1.95111567e-03 -1.68878466e-01 -1.05658814e-01 -2.38340721e-02  6.14980869e-02  7.95577615e-02 -1.5707267

In [77]:
class PredictionModel(nnx.Module):
    """Model to predict action probabilities and reward of the current state."""
    state_embedder: Connect4StateEmbedder
    action_embedder: Connect4ActionEmbedder

    embedding_dim: int = 64
    num_actions: int = 7

    def __init__(self, state_embedder: Connect4StateEmbedder, action_embedder: Connect4ActionEmbedder, *, rngs: nnx.Rngs):
        self.state_embedder = state_embedder
        self.action_embedder = action_embedder
        self.reward_head = nnx.Linear(self.embedding_dim, 1, rngs=rngs)

    def action_logits(self, state_batch: jax.Array) -> jax.Array:
        state_embeddings = self.state_embedder(state_batch)
        all_action_embeddings = self.action_embedder.all_action_embeddings()
        logits = state_embeddings @ all_action_embeddings.T
        return logits
    
    def action_probs(self, state_batch: jax.Array) -> jax.Array:
        logits = self.action_logits(state_batch)
        return jax.nn.softmax(logits)
    
    def reward_pred(self, state_batch: jax.Array) -> jax.Array:
        state_embeddings = self.state_embedder(state_batch)
        return self.reward_head(state_embeddings)



In [78]:
def test_state_action_model():
    state_embedder = Connect4StateEmbedder(rngs=nnx.Rngs(0))
    action_embedder = Connect4ActionEmbedder(rngs=nnx.Rngs(1))
    state_action_model = PredictionModel(state_embedder, action_embedder, rngs=nnx.Rngs(2))
    print('action_logits:\n', state_action_model.action_logits(state_batch[:2]))
    print('\naction_probs:\n', state_action_model.action_probs(state_batch[:2]))
    print('\nreward_pred:\n', state_action_model.reward_pred(state_batch[:2]))

test_state_action_model()


action_logits:
 [[ 0.          0.          0.          0.          0.          0.          0.        ]
 [-0.05365274  0.04191583 -0.05556455 -0.0902497  -0.04111161  0.00694049  0.01304062]]

action_probs:
 [[0.14285715 0.14285715 0.14285715 0.14285715 0.14285715 0.14285715 0.14285715]
 [0.13876377 0.15267958 0.13849871 0.13377723 0.14051497 0.14743187 0.14833395]]

reward_pred:
 [[ 0.        ]
 [-0.06306502]]


# Train Model

In [79]:
import optax

@nnx.jit
def l2_loss(model, alpha=0.001):
    loss = sum(
        (alpha * (w ** 2).sum()) 
        for w in jax.tree.leaves(nnx.state(model, nnx.Param))
    )
    return loss

@nnx.jit
def loss_fn(prediction_model: PredictionModel, batch, l2_weight: float = 1e-4):
    action_logits = prediction_model.action_logits(batch['state'])
    action_labels = batch['action']-1
    action_data_loss = optax.softmax_cross_entropy_with_integer_labels(logits=action_logits, labels=action_labels).mean()

    reward_pred = prediction_model.reward_pred(batch['state'])
    reward_labels = batch['reward']
    reward_data_loss = ((reward_labels - reward_pred.squeeze())**2).mean()

    parameter_loss_l2 = l2_weight * l2_loss(prediction_model)
    
    total_loss = action_data_loss + reward_data_loss + parameter_loss_l2
    return total_loss, action_logits

@nnx.jit
def loss_fn_2(prediction_model: PredictionModel, batch, l2_weight: float = 1e-4):
    action_logits = prediction_model.action_logits(batch['state'])
    action_labels = batch['action']-1
    action_data_loss = optax.softmax_cross_entropy_with_integer_labels(logits=action_logits, labels=action_labels).mean()

    reward_pred = prediction_model.reward_pred(batch['state'])
    reward_labels = batch['reward']
    reward_data_loss = ((reward_labels - reward_pred.squeeze())**2).mean()

    parameter_loss_l2 = l2_weight * l2_loss(prediction_model)
    
    total_loss = action_data_loss + reward_data_loss + parameter_loss_l2
    return total_loss, (action_logits, reward_pred)



In [80]:
def test_loss_fn():
    state_embedder = Connect4StateEmbedder(rngs=nnx.Rngs(0))
    action_embedder = Connect4ActionEmbedder(rngs=nnx.Rngs(1))
    prediction_model = PredictionModel(state_embedder, action_embedder, rngs=nnx.Rngs(2))
    loss, (action_logits, reward_pred) = loss_fn_2(prediction_model, batch_2)
    print('loss: ', loss)
    print('action_logits: ', action_logits)
    print('reward_pred: ', reward_pred)

with jax.disable_jit():
    test_loss_fn()


# loss:  1.927142
# logits:  [[ 0.          0.          0.          0.          0.          0.          0.        ]
#  [-0.05365274  0.04191583 -0.05556455 -0.0902497  -0.04111161  0.00694049  0.01304064]]

loss:  1.9291308
action_logits:  [[ 0.          0.          0.          0.          0.          0.          0.        ]
 [-0.05365274  0.04191583 -0.05556455 -0.0902497  -0.04111161  0.00694049  0.01304062]]
reward_pred:  [[ 0.        ]
 [-0.06306502]]


In [106]:
@nnx.jit
def train_step(prediction_model: PredictionModel, optimizer: nnx.Optimizer, metrics: nnx.MultiMetric, batch):
  """Train for a single step."""
  grad_fn = nnx.value_and_grad(loss_fn_2, has_aux=True)
  (loss, (logits, reward_pred)), grads = grad_fn(prediction_model, batch)
  metrics.update(loss=loss,
    logits=logits, labels=batch['action']-1,
    reward_pred=reward_pred, reward_labels=batch['reward'])  # In-place updates.
  optimizer.update(grads)  # In-place updates.

@nnx.jit
def eval_step(prediction_model: PredictionModel, metrics: nnx.MultiMetric, batch):
  loss, (logits, reward_pred) = loss_fn_2(prediction_model, batch)
  metrics.update(loss=loss, logits=logits, labels=batch['action'])  # In-place updates.


In [130]:
from flax import nnx
import jax.numpy as jnp
from abc import ABC, abstractmethod
from typing_extensions import override

class TwoValueAverageMetric(nnx.Metric, ABC):
    total: nnx.metrics.MetricState
    count: nnx.metrics.MetricState

    def __init__(self, argname_1: str, argname_2: str):
        self.argname_1 = argname_1
        self.argname_2 = argname_2
        self.total = nnx.metrics.MetricState(jnp.array(0, dtype=jnp.float32))
        self.count = nnx.metrics.MetricState(jnp.array(0, dtype=jnp.int32))

    def reset(self) -> None:
        self.total.value = jnp.array(0, dtype=jnp.float32)
        self.count.value = jnp.array(0, dtype=jnp.int32)

    def update(self, **kwargs) -> None:
        if self.argname_1 not in kwargs:
            raise TypeError(f"Expected keyword argument '{self.argname_1}'")
        if self.argname_2 not in kwargs:
            raise TypeError(f"Expected keyword argument '{self.argname_2}'")      
        v1, v2 = kwargs[self.argname_1], kwargs[self.argname_2]
        self.update_pair(v1, v2)

    @abstractmethod
    def update_pair(v1: jax.Array, v2: jax.Array) -> None:
        raise NotImplementedError("Must be implemented by subclass")

    @abstractmethod
    def compute(self) -> jax.Array:
        raise NotImplementedError("Must be implemented by subclass")

class MeanSquaredError(TwoValueAverageMetric):
    @override
    def __init__(self, argname_1:str='mse_1', argname_2:str='mse_2'):
        super().__init__(argname_1, argname_2)

    @override
    def update_pair(self, v1: jax.Array, v2: jax.Array) -> None:
        if v1.shape != v2.shape:
            raise ValueError(f"Expected shapes {v1.shape} and {v2.shape} to be equal")
        self.total.value += ((v1 - v2)**2).sum()
        self.count.value += v1.size

    def compute(self) -> jax.Array:
        """Compute and return the average."""
        return self.total.value / self.count.value

class LogitAccuracy(TwoValueAverageMetric):
    @override
    def __init__(self, argname_1:str='logits', argname_2:str='labels'):
        super().__init__(argname_1, argname_2)

    @override
    def update_pair(self, logits: jax.Array, labels: jax.Array) -> None:
        if logits.ndim != labels.ndim + 1:
            raise ValueError(f'Expected logits.ndim==labels.ndim+1, got {logits.ndim} and {labels.ndim}')

        self.total.value += (logits.argmax(axis=-1) == labels).sum()
        self.count.value += labels.size

    def compute(self) -> jax.Array:
        """Compute and return the average."""
        return self.total.value / self.count.value






def test_mean_squared_error():
    mse = MeanSquaredError('reward_true', 'reward_pred')
    mse.update(reward_true=jnp.array([1, 2, 3]), reward_pred=jnp.array([1, 2, 5]))
    print(mse.compute())
    mse.reset()
    print(mse.compute())

test_mean_squared_error()

def test_multi_metric():
    metrics = nnx.MultiMetric(
        accuracy=nnx.metrics.Accuracy(),
        action_accuracy=nnx.metrics.Accuracy("action_logits","action_labels"),
        reward_mse=MeanSquaredError('reward_true', 'reward_pred'),
        loss=nnx.metrics.Average('loss'))
    # metrics = nnx.MultiMetric(action=nnx.metrics.Accuracy(), loss=nnx.metrics.Average('loss'))
    metrics.update(
        loss=1.0,
        logits=jnp.array([[1, 2, 3], [4, 5, 6]]),
        labels=jnp.array([1, 2]),
        action_logits=jnp.array([[1, 2, 3], [4, 5, 6]]),
        action_labels=jnp.array([1, 2]),
        reward_true=jnp.array([1, 2, 3]),
        reward_pred=jnp.array([1, 2, 5]))
    print(metrics.compute())

test_multi_metric()


1.3333334
nan
{'action': Array(0.5, dtype=float32), 'reward_mse': Array(1.3333334, dtype=float32), 'loss': Array(1., dtype=float32)}


In [109]:
def test_train_step():
    state_embedder = Connect4StateEmbedder(rngs=nnx.Rngs(10))
    action_embedder = Connect4ActionEmbedder(rngs=nnx.Rngs(1))
    prediction_model = PredictionModel(state_embedder, action_embedder, rngs=nnx.Rngs(2))
    optimizer = nnx.Optimizer(prediction_model, optax.adamw(learning_rate=0.0005))
    # metrics = nnx.MultiMetric(action_accuracy=nnx.metrics.Accuracy(), reward_mse=MeanSquaredError(), loss=nnx.metrics.Average('loss'))
    metrics = nnx.MultiMetric(action_accuracy=nnx.metrics.Accuracy(), reward_mse=nnx.metrics.Welford(), loss=nnx.metrics.Average('loss'))
    # metrics = nnx.MultiMetric(action_accuracy=nnx.metrics.Accuracy(), loss=nnx.metrics.Average('loss'))
    
    state_action_counts = {}    
    for i in range(4):
        state = batch_full['state'][i]
        state_action_counts[i] = Counter(a.item() for s,a in zip(batch_full['state'], batch_full['action']) if (s == state).all())

    for i in range(400+1):
        train_step(prediction_model, optimizer, metrics, batch_full)
        if i % 200 == 0:
            print(f'train_step: {i}: {metrics.compute()}')
            for i in range(4):                
                state = batch_full['state'][i]
                action_probs = prediction_model.action_probs(jnp.array([state]))
                print(i, jnp.array_str(action_probs * 100, precision=4, max_line_width=100, suppress_small=True), state_action_counts[i])
            print()

test_train_step()

# train_step: 0: {'accuracy': Array(0.14363885, dtype=float32), 'loss': Array(1.9785088, dtype=float32)}
# 0 [[14.2948 14.2847 14.241  14.3802 14.2482 14.3175 14.2335]] Counter({1: 23, 5: 16, 4: 14, 6: 13, 3: 12, 2: 12, 7: 10})
# 1 [[14.9753 14.4342 14.0819 15.6058 13.1944 14.3685 13.3399]] Counter({7: 3, 6: 3, 3: 3, 5: 2, 1: 1, 2: 1})
# 2 [[15.3895 15.3645 12.9772 16.6033 13.0454 13.8517 12.7683]] Counter({4: 1, 3: 1, 1: 1})
# 3 [[15.1959 15.988  12.1962 19.0648 12.5115 13.4398 11.6039]] Counter({5: 1})

# train_step: 200: {'accuracy': Array(0.6671386, dtype=float32), 'loss': Array(0.9482211, dtype=float32)}
# 0 [[22.9432 12.1785 11.9234 14.0172 15.9442 12.9932 10.0002]] Counter({1: 23, 5: 16, 4: 14, 6: 13, 3: 12, 2: 12, 7: 10})
# 1 [[ 6.2996  7.7686 24.5144  0.1851 14.6386 22.1654 24.4284]] Counter({7: 3, 6: 3, 3: 3, 5: 2, 1: 1, 2: 1})
# 2 [[33.8826  1.2373 30.2037 25.751   0.9681  1.9616  5.9957]] Counter({4: 1, 3: 1, 1: 1})
# 3 [[27.6696  0.2975  0.3806 17.6307 52.2947  1.326   0.4009]] Counter({5: 1})

# train_step: 400: {'accuracy': Array(0.7840295, dtype=float32), 'loss': Array(0.5941784, dtype=float32)}
# 0 [[22.3311 12.1671 12.2211 14.1951 16.2631 12.9815  9.8409]] Counter({1: 23, 5: 16, 4: 14, 6: 13, 3: 12, 2: 12, 7: 10})
# 1 [[ 7.1177  7.8012 24.0158  0.0328 15.7439 23.1178 22.1708]] Counter({7: 3, 6: 3, 3: 3, 5: 2, 1: 1, 2: 1})
# 2 [[31.2154  0.1368 34.4281 33.3913  0.0374  0.2569  0.5341]] Counter({4: 1, 3: 1, 1: 1})
# 3 [[ 1.8426  0.0026  0.0019  1.4788 96.6549  0.0186  0.0006]] Counter({5: 1})

TypeError: Expected keyword argument 'values'

In [ ]:
# TODO: How do we stream a bunch of different batches?
# train_batches = [(i, batch) for i in range(2000)]
# test_batches = [(i, batch) for i in range(100)]

#train_batches = [(i, batch_repeated) for i in range(2000)]
#test_batches = [(i, batch_repeated) for i in range(100)]

# NUM_TRAIN_BATCHES = 2000
NUM_TRAIN_BATCHES = 1
train_batches = [(i, batch_full) for i in range(NUM_TRAIN_BATCHES)]
test_batches = [(i, batch_full) for i in range(100)]


metrics_history = {
  'train_loss': [],
  'train_accuracy': [],
  'test_loss': [],
  'test_accuracy': [],
}

train_steps = 10000
eval_every = 200
batch_size = 32

# for step, batch in enumerate(train_ds.as_numpy_iterator()):
for step, batch in train_batches:
  # with jax.disable_jit():
  #   train_step(state_embedder, action_embedder, optimizer, metrics, batch)
  train_step(state_embedder, action_embedder, optimizer, metrics, batch)

  if step > 0 and (step % eval_every == 0 or step == train_steps - 1):  # One training epoch has passed.
    # Log the training metrics.
    for metric, value in metrics.compute().items():  # Compute the metrics.
      metrics_history[f'train_{metric}'].append(value)  # Record the metrics.
    metrics.reset()  # Reset the metrics for the test set.

    # Compute the metrics on the test set after each training epoch.
    for _, test_batch in test_batches:
      eval_step(state_embedder, action_embedder, metrics, test_batch)

    # Log the test metrics.
    for metric, value in metrics.compute().items():
      metrics_history[f'test_{metric}'].append(value)
    metrics.reset()  # Reset the metrics for the next training epoch.

    print(
      f"[train] step: {step}, "
      f"loss: {metrics_history['train_loss'][-1]}, "
      f"accuracy: {metrics_history['train_accuracy'][-1] * 100}"
    )
    # print(
    #   f"[test] step: {step}, "
    #   f"loss: {metrics_history['test_loss'][-1]}, "
    #   f"accuracy: {metrics_history['test_accuracy'][-1] * 100}"
    # )


# ## Outputs with embedding normalization.
# [train] step: 200, loss: 1.4493087530136108, accuracy: 41.806243896484375
# [train] step: 400, loss: 0.495466947555542, accuracy: 77.71910858154297
# [train] step: 600, loss: 0.4014337956905365, accuracy: 80.45576477050781
# [train] step: 800, loss: 0.3858652412891388, accuracy: 80.7986831665039
# [train] step: 1000, loss: 0.38055431842803955, accuracy: 80.97492218017578
# [train] step: 1200, loss: 0.37459635734558105, accuracy: 81.28226470947266
# [train] step: 1400, loss: 0.3812824487686157, accuracy: 81.351806640625
# [train] step: 1600, loss: 0.366107702255249, accuracy: 81.67510986328125
# [train] step: 1800, loss: 0.3646901547908783, accuracy: 81.75855255126953
# [train] step: 2000, loss: 0.3648184537887573, accuracy: 81.75786590576172
# [train] step: 2200, loss: 0.35771697759628296, accuracy: 82.1707763671875
# [train] step: 2400, loss: 0.3564577102661133, accuracy: 82.21227264404297
# [train] step: 2600, loss: 0.3553982973098755, accuracy: 82.25535583496094
# [train] step: 2800, loss: 0.3551204204559326, accuracy: 82.25627136230469
# [train] step: 3000, loss: 0.35472187399864197, accuracy: 82.2571792602539
# [train] step: 3200, loss: 0.35463690757751465, accuracy: 82.25672912597656
# [train] step: 3400, loss: 0.35420480370521545, accuracy: 82.25741577148438
# [train] step: 3600, loss: 0.35807478427886963, accuracy: 82.2020034790039
# [train] step: 3800, loss: 0.35097551345825195, accuracy: 82.39557647705078
# [train] step: 4000, loss: 0.3513658940792084, accuracy: 82.39649200439453
# [train] step: 4200, loss: 0.3515341877937317, accuracy: 82.39580535888672
# [train] step: 4400, loss: 0.35152217745780945, accuracy: 82.39762878417969
# [train] step: 4600, loss: 0.35154837369918823, accuracy: 82.39649200439453
# [train] step: 4800, loss: 0.3515065014362335, accuracy: 82.40766143798828
# [train] step: 5000, loss: 0.3419311046600342, accuracy: 83.08686828613281
# ...
# [train] step: 19200, loss: 0.21814590692520142, accuracy: 90.15048217773438
# [train] step: 19400, loss: 0.2181217074394226, accuracy: 90.15048217773438
# [train] step: 19600, loss: 0.21812711656093597, accuracy: 90.15048217773438
# [train] step: 19800, loss: 0.21816301345825195, accuracy: 90.14934539794922


# logits: [[-0.12943102 -0.18300387 -0.31407255 -0.26985747 -0.03317889  0.8657205  -0.06745078]
#          [-0.3108766  -0.11808072 -0.00877056 -0.12999986 -0.18233624 -0.03913596   0.86498916]]
# probs: [[0.11739378 0.11127014 0.09760144 0.10201374 0.12925485 0.31756592  0.1249001 ]
#         [0.09565745 0.11599759 0.12939627 0.1146232  0.10877853 0.12552617  0.31002077]]


In [ ]:
# Compare action counts to action probabilities.
for i in range(10):
    state = batch_full['state'][i]
    counts = Counter(a.item() for s,a in zip(batch_full['state'], batch_full['action']) if (s == state).all())
    probs = state_action_model.probs(jnp.array([state]))
    print(i, jnp.array_str(probs * 100, precision=4, max_line_width=100, suppress_small=True), counts)

## No training.

# 0 [[16.0473 15.4123 13.6238 13.4007 12.6302 15.335  13.5506]] Counter({1: 23, 5: 16, 4: 14, 6: 13, 3: 12, 2: 12, 7: 10})
# 1 [[29.2187 19.2536 10.1736  7.9687  5.0374 18.8691  9.4789]] Counter({7: 3, 6: 3, 3: 3, 5: 2, 1: 1, 2: 1})
# 2 [[39.9711 18.2044  6.5298  5.4635  2.1863 20.3188  7.3262]] Counter({4: 1, 3: 1, 1: 1})
# 3 [[48.2277 17.6604  3.9698  3.1657  1.1044 21.3496  4.5223]] Counter({5: 1})
# 4 [[52.5742 19.8617  1.9912  1.4634  0.372  21.7475  1.9901]] Counter({5: 1})
# 5 [[59.2366 18.4123  1.1171  0.8116  0.1682 19.1115  1.1429]] Counter({7: 1})
# 6 [[62.148  21.6044  0.7341  0.471   0.061  14.4665  0.515 ]] Counter({1: 1})
# 7 [[63.5058 21.5263  0.4813  0.303   0.0317 13.7824  0.3696]] Counter({7: 1})
# 8 [[76.6878 13.1025  0.1152  0.0922  0.006   9.8978  0.0985]] Counter({1: 1})
# 9 [[79.2133 11.9533  0.0701  0.0548  0.0032  8.6434  0.0619]] Counter({1: 1})